In [20]:
from datetime import datetime
from anthropic import Anthropic
from IPython.display import Markdown
from dotenv import load_dotenv

load_dotenv()
model = "claude-3-5-haiku-latest"
client = Anthropic()
num_tokens = 1000

In [21]:
def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
   if not date_format:
      raise ValueError(f"{date_format} cannot be empty.")
   
   return datetime.now().strftime(date_format)

get_current_datetime_schema = {
   "name": "get_current_datetime",
   "description": "Get the current date and time formatted according to a specified pattern. Returns the current datetime as a formatted string.",
   "input_schema": {
     "type": "object",
     "properties": {
       "date_format": {
         "type": "string",
         "description": "Python strftime format string for datetime formatting. Common patterns: '%Y-%m-%d' for date only (e.g., 2025-10-13), '%H:%M:%S' for time only (e.g., 14:30:45), '%Y-%m-%d %H:%M:%S' for full datetime (e.g., 2025-10-13 14:30:45), '%B %d, %Y' for readable date (e.g., October 13, 2025). Cannot be empty. Defaults to '%Y-%m-%d %H:%M:%S' if not provided.",
         "default": "%Y-%m-%d %H:%M:%S"
       },
     },
     "required": []
   },
}

In [44]:
# Main Helper Functions
def add_user_message(messages, text):
   user_message = {
      "role": "user",
      "content": text
   }
   messages.append(user_message)

   response = client.messages.create(
      model=model,
      max_tokens=num_tokens,
      messages=messages,
      tools=[get_current_datetime_schema]
   )

def add_assistant_message(messages, text):
   assistant_message = {
      "role": "assistant",
      "content": text
   }
   messages.append(assistant_message)

def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
   params = {
      "model": model,
      "max_tokens": num_tokens,
      "messages": messages,
      "temperature": temperature
   }

   if system:
      params["system"] = system

   with client.messages.stream(**params) as stream:
      for text in stream.text_stream:
         print(text, end="")
         pass
   response = stream.get_final_message()
   # message = client.messages.create(**params)
   return response.content[0].text

In [22]:
messages = []

messages.append(
   {
      "role": "user",
      "content": "What is the exact time, formatted as HH:MM:SS?"
   }
)

response = client.messages.create(
   model=model,
   max_tokens=num_tokens,
   messages=messages,
   tools=[get_current_datetime_schema],
)

messages.append(
   {
      "role": "assistant",
      "content": response.content
   }
)
messages

[{'role': 'user', 'content': 'What is the exact time, formatted as HH:MM:SS?'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll retrieve the current time for you using the specified format.", type='text'),
   ToolUseBlock(id='toolu_01WohDiDtZmKVjEG5ArXoNLj', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]}]

In [12]:
response

Message(id='msg_012xaGB8uTWoGLNWJ6Bv7vFC', content=[TextBlock(citations=None, text="I'll retrieve the current time for you using the specified format.", type='text'), ToolUseBlock(id='toolu_018FXUReaBtpwnaR7uNuGaS9', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=517, output_tokens=76, server_tool_use=None, service_tier='standard', cache_creation={'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}))

In [13]:
response.content[1]

ToolUseBlock(id='toolu_018FXUReaBtpwnaR7uNuGaS9', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')

In [14]:
response.content[1].input

{'date_format': '%H:%M:%S'}

In [23]:
wanted = get_current_datetime(**response.content[1].input)

In [24]:
messages.append({
   "role": "user",
   "content": [{
      "type": "tool_result",
      "tool_use_id": response.content[1].id,
      "content": wanted,
      "is_error": False
   }]
})

client.messages.create(
   model=model,
   max_tokens=num_tokens,
   messages=messages,
   tools=[get_current_datetime_schema]
)

Message(id='msg_01GH8fHGM4vPDC5SuENEJrZZ', content=[TextBlock(citations=None, text='The current time is 21:51:25 (in 24-hour format).', type='text')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=609, output_tokens=23, server_tool_use=None, service_tier='standard', cache_creation={'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}))

Note: 
You need to import datetime FROM datetime to use datetime.now()